In [14]:
# 1. Imports & configuración
import sys
import os

# Añadimos la carpeta raíz al sistema para poder importar desde 'src'
sys.path.append(os.path.abspath(os.path.join('..')))

# Dependencias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Importar funciones propias
from src.loader import load_and_clean_data, filter_by_period
from src.metrics import calculate_returns, get_global_stats, analyze_by_symbol

In [15]:
# 2. Carga de datos
# 2.1 Leer el archivo CSV (‘symbol’, ‘date’, ‘open’, ‘high’, ‘low’, ‘close’, ‘volume’)
# 2.2 Convertir date al tipo datetime
ruta_csv = '../data/S&P_500_Stock_Prices_2014-2017.csv'
df = load_and_clean_data(ruta_csv)

# 2.3 Mostrar df.head(), df.info(), df.describe() para entender estructura y tipos
print("\n- Primeras filas -")
display(df.head())

print("- Información del Dataset -")
df.info()

print("- Estadísticas Descriptivas -")
display(df.describe())


- Primeras filas -


,symbol,date,open,high,low,close,volume
57,A,2014-01-02,57.10,57.100,56.15,56.21,1916160
540,A,2014-01-03,56.39,57.345,56.26,56.92,1866651
1023,A,2014-01-06,57.40,57.700,56.56,56.64,1777472
1506,A,2014-01-07,56.95,57.630,56.93,57.45,1463208
1989,A,2014-01-08,57.33,58.540,57.17,58.39,2659468


- Información del Dataset -
<class 'pandas.core.frame.DataFrame'>
Index: 497461 entries, 57 to 497471
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   symbol  497461 non-null  object        
 1   date    497461 non-null  datetime64[ns]
 2   open    497461 non-null  float64       
 3   high    497461 non-null  float64       
 4   low     497461 non-null  float64       
 5   close   497461 non-null  float64       
 6   volume  497461 non-null  int64         
dtypes: datetime64[ns](1), float64(4), int64(1), object(1)
memory usage: 30.4+ MB
- Estadísticas Descriptivas -


,date,open,high,low,close,volume
count,497461,497461.000000,497461.000000,497461.000000,497461.000000,4.974610e+05
mean,2016-01-06 17:16:56.524310784,86.352275,87.132717,85.552616,86.368586,4.253695e+06
min,2014-01-02 00:00:00,1.620000,1.690000,1.500000,1.590000,1.010000e+02
25%,2015-01-08 00:00:00,41.690000,42.090000,41.280000,41.700000,1.080183e+06
50%,2016-01-11 00:00:00,64.970000,65.560000,64.357400,64.980000,2.085013e+06
75%,2017-01-06 00:00:00,98.410000,99.230000,97.580000,98.420000,4.271999e+06
max,2017-12-29 00:00:00,2044.000000,2067.990000,2035.110000,2049.000000,6.182376e+08
std,NaN,101.471228,102.312340,100.571231,101.471516,8.232210e+06


In [16]:
# 3. Limpieza y preparación de datos
# 3.1 Verificar valores nulos o datos faltantes (Ya lo hace load_and_clean_data internamente)
# 3.2 Asegurar que close, open, volume están como tipos numéricos (Hecho en loader)
# 3.3 Ordenar los datos por symbol y date (Hecho en loader)

# 3.4 - 3.6 Crear columnas nuevas útiles: Rendimiento diario simple y Rendimiento acumulado
df = calculate_returns(df)

# 3.7 Filtrar fechas, símbolos o periodos para análisis
# Ejemplo 1: Filtrar por periodo (2015) (usando la función que creamos en loader.py)
df_2015 = filter_by_period(df, '2015-01-01', '2015-12-31')
print(f"Registros en 2015: {len(df_2015)}")

# Ejemplo 2: Filtrar por un símbolo específico
df_apple = df[df['symbol'] == 'AAPL']
print(f"Registros de AAPL: {len(df_apple)}")

display(df_apple.head())

Registros en 2015: 123713
Registros de AAPL: 1007


,symbol,date,open,high,low,close,volume,return_daily,return_accumulated
1,AAPL,2014-01-02,79.3828,79.5756,78.8601,79.0185,58791957,NaN,1.000000
484,AAPL,2014-01-03,78.9799,79.0999,77.2042,77.2828,98303870,-0.021966,0.978034
967,AAPL,2014-01-06,76.7785,78.1142,76.2285,77.7042,103359151,0.005453,0.983367
1450,AAPL,2014-01-07,77.7599,77.9942,76.8464,77.1481,79432766,-0.007157,0.976330
1933,AAPL,2014-01-08,76.9728,77.9371,76.9556,77.6371,64686685,0.006338,0.982518


In [17]:
# 4. Análisis exploratorio

# 4.1 Estadísticas globales
# 4.1.1 Número total de registros, número de símbolos (symbol)
# 4.1.2 Estadísticas de rendimiento diario (return_daily): media, mediana, desviación estándar
# 4.1.3 Volumen medio de negociación
global_stats = get_global_stats(df)
print("- Estadísticas globales -")
for key, val in global_stats.items():
    print(f"{key}: {val}")

# 4.2 Distribución de rendimientos por símbolo
# 4.2.1 - 4.2.4 Agrupar por symbol para calcular: rendimiento medio diario, volatilidad diaria, crecimiento acumulado
# 4.2.5 Ordenar símbolos por rendimiento acumulado
summary = analyze_by_symbol(df)
print("\n- Top 10 Símbolos por rendimiento acumuilado -")
display(summary.head(10))

- Estadísticas globales -
total_records: 497461
num_symbols: 505
mean_return: 0.00046015953927684244
median_return: 0.0005812263876781021
standar_deviation: 0.016285337426885725
mean_volume: 4253694.672418541

- Top 10 Símbolos por rendimiento acumuilado -


,mean_return,volatility,final_growth
symbol,,,
NVDA,0.002760,0.023739,12.200504
AVGO,0.001780,0.020237,4.879392
EA,0.001692,0.018899,4.601840
ALGN,0.001545,0.019607,3.900808
NFLX,0.001651,0.026585,3.703546
ATVI,0.001419,0.018718,3.504151
LUV,0.001402,0.018171,3.466631
SWKS,0.001488,0.022496,3.465328
EW,0.001395,0.018598,3.419600


In [ ]:
# 5. Uso de NumPy para análisis auxiliar

# Preparación: Elegimos una empresa (ejemplo: Apple) y sacamos sus precios a una lista
simbolo = 'AAPL'
precios = df[df['symbol'] == simbolo]['close'].values

# 5.1 Usar np.diff() + división para calcular rendimientos simples manualmente
# Precio hoy - Precio ayer
diferencia_precios = np.diff(precios)         
# Precios desde el primero hasta el penúltimo
precio_inicial = precios[:-1]         
        
rendimientos_manuales = diferencia_precios / precio_inicial

print(f"- Análisis con NumPy para {simbolo} -")
print(f"Primeros 3 rendimientos: {rendimientos_manuales[:3]}")

# 5.2 Calcular percentiles de rendimiento diario (Percentil 95)
# El percentil 95 es que supera al 95% de las empresas / top 5%
percentil_95 = np.percentile(rendimientos_manuales, 95)

print(f"Percentil 95: {percentil_95}")
print(f"Esto significa que solo el 5% de los días se ganó más de un {percentil_95 * 100}%")

# 5.3 Usar np.cumprod() para calcular crecimiento de inversión
# Convertimos rendimientos (ej. 0.05 a 1.05) para poder multiplicarlos
factores_crecimiento = 1 + rendimientos_manuales
crecimiento_acumulado = np.cumprod(factores_crecimiento)

# El último valor nos dice cuánto creció 1€ desde el inicio hasta el final
print(f"Resultado final de invertir 1€ (2014-2017): {crecimiento_acumulado[-1]:.2f}€")

- Análisis con NumPy para AAPL -
Primeros 3 rendimientos: [-0.02196574  0.0054527  -0.00715663]
Percentil 95: 0.02175276824011163
Esto significa que solo el 5% de los días se ganó más de un 2.175276824011163 %
Resultado final de invertir 1€: 2.14€
